# Redis を使用したエンベディング検索

このノートブックでは、データをダウンロードし、エンベディングを作成し、その後複数のベクトルデータベースを使用してインデックス化と検索を行う簡単なフローを説明します。これは、チャットボット、トピックモデリングなどの本番環境のユースケースをサポートするために、セキュアな環境で独自のデータと共にエンベディングを保存・検索したいお客様にとって一般的な要件です。

### ベクトルデータベースとは

ベクトルデータベースは、エンベディングベクトルを保存、管理、検索するために作られたデータベースです。非構造化データ（テキスト、音声、動画など）をベクトルとしてエンコードし、機械学習モデルで利用するエンベディングの使用は、自然言語、画像認識、その他の非構造化データ形式に関わるユースケースでのAIの有効性が高まったことにより、近年爆発的に増加しています。ベクトルデータベースは、企業がこれらのユースケースを提供し、スケールするための効果的なソリューションとして登場しました。

### なぜベクトルデータベースを使用するのか

ベクトルデータベースにより、企業はこのリポジトリで共有している多くのエンベディングのユースケース（質問応答、チャットボット、推薦サービスなど）を、セキュアでスケーラブルな環境で活用できるようになります。多くのお客様は小規模でエンベディングを使用して問題を解決していますが、パフォーマンスとセキュリティの問題により本番環境への移行が阻まれています。私たちはベクトルデータベースをその解決の重要な要素と考えており、このガイドではテキストデータのエンベディング、ベクトルデータベースへの保存、セマンティック検索での使用の基本について説明します。

### デモフロー
デモフローは以下の通りです：
- **セットアップ**: パッケージをインポートし、必要な変数を設定
- **データ読み込み**: データセットを読み込み、OpenAIエンベディングを使用してエンベディングを作成
- **Redis**
    - *セットアップ*: Redis-Pyクライアントを設定。詳細は[こちら](https://github.com/redis/redis-py)を参照
    - *データのインデックス化*: 利用可能なすべてのフィールドでベクトル検索とハイブリッド検索（ベクトル + 全文検索）用の検索インデックスを作成
    - *データ検索*: 様々な目的を念頭に置いたサンプルクエリをいくつか実行

このノートブックを実行することで、ベクトルデータベースのセットアップと使用方法の基本的な理解を得ることができ、エンベディングを活用したより複雑なユースケースに進むことができます。

## セットアップ

必要なライブラリをインポートし、使用したい埋め込みモデルを設定します。

In [ ]:
# We'll need to install the Redis client
!pip install redis

#Install wget to pull zip file
!pip install wget

In [1]:
import openai

from typing import List, Iterator
import pandas as pd
import numpy as np
import os
import wget
from ast import literal_eval

# Redis client library for Python
import redis

# I've set this to our new embeddings model, this can be changed to the embedding model of your choice
EMBEDDING_MODEL = "text-embedding-3-small"

# Ignore unclosed SSL socket warnings - optional in case you get these errors
import warnings

warnings.filterwarnings(action="ignore", message="unclosed", category=ResourceWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning) 

## データの読み込み

このセクションでは、このセッションの前に準備した埋め込みデータを読み込みます。

In [ ]:
embeddings_url = 'https://cdn.openai.com/API/examples/data/vector_database_wikipedia_articles_embedded.zip'

# The file is ~700 MB so this will take some time
wget.download(embeddings_url)

In [ ]:
import zipfile
with zipfile.ZipFile("vector_database_wikipedia_articles_embedded.zip","r") as zip_ref:
    zip_ref.extractall("../data")

In [2]:
article_df = pd.read_csv('../data/vector_database_wikipedia_articles_embedded.csv')

In [3]:
article_df.head()

,id,url,title,text,title_vector,content_vector,vector_id
0,1,https://simple.wikipedia.org/wiki/April,April,April is the fourth month of the year in the J...,"[0.001009464613161981, -0.020700545981526375, ...","[-0.011253940872848034, -0.013491976074874401,...",0
1,2,https://simple.wikipedia.org/wiki/August,August,August (Aug.) is the eighth month of the year ...,"[0.0009286514250561595, 0.000820168002974242, ...","[0.0003609954728744924, 0.007262262050062418, ...",1
2,6,https://simple.wikipedia.org/wiki/Art,Art,Art is a creative activity that expresses imag...,"[0.003393713850528002, 0.0061537534929811954, ...","[-0.004959689453244209, 0.015772193670272827, ...",2
3,8,https://simple.wikipedia.org/wiki/A,A,A or a is the first letter of the English alph...,"[0.0153952119871974, -0.013759135268628597, 0....","[0.024894846603274345, -0.022186409682035446, ...",3
4,9,https://simple.wikipedia.org/wiki/Air,Air,Air refers to the Earth's atmosphere. Air is a...,"[0.02224554680287838, -0.02044147066771984, -0...","[0.021524671465158463, 0.018522677943110466, -...",4


In [4]:
# Read vectors from strings back into a list
article_df['title_vector'] = article_df.title_vector.apply(literal_eval)
article_df['content_vector'] = article_df.content_vector.apply(literal_eval)

# Set vector_id to be a string
article_df['vector_id'] = article_df['vector_id'].apply(str)

In [5]:
article_df.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              25000 non-null  int64 
 1   url             25000 non-null  object
 2   title           25000 non-null  object
 3   text            25000 non-null  object
 4   title_vector    25000 non-null  object
 5   content_vector  25000 non-null  object
 6   vector_id       25000 non-null  object
dtypes: int64(1), object(6)
memory usage: 1.3+ MB


# Redis

このチュートリアルで取り上げる次のベクターデータベースは**[Redis](https://redis.io)**です。おそらくあなたはすでにRedisをご存知でしょう。あなたが知らないかもしれないのは[RediSearchモジュール](https://github.com/RediSearch/RediSearch)です。企業は何年もの間、すべての主要なクラウドプロバイダー、Redis Cloud、そしてオンプレミスでRediSearchモジュールと組み合わせてRedisを使用してきました。最近、RedisチームはRediSearchが既に持っていた機能に加えて、このモジュールにベクター保存と検索機能を追加しました。

Redisの大きなエコシステムを考えると、あなたが必要とする言語のクライアントライブラリがほぼ確実に存在します。RediSearchコマンドを実行するために任意の標準的なRedisクライアントライブラリを使用できますが、RediSearch APIをラップするライブラリを使用するのが最も簡単です。以下にいくつかの例を示しますが、より多くのクライアントライブラリは[こちら](https://redis.io/resources/clients/)で見つけることができます。

| プロジェクト | 言語 | ライセンス | 作者 | Stars |
|----------|---------|--------|---------|-------|
| [jedis][jedis-url] | Java | MIT | [Redis][redis-url] | ![Stars][jedis-stars] |
| [redis-py][redis-py-url] | Python | MIT | [Redis][redis-url] | ![Stars][redis-py-stars] |
| [node-redis][node-redis-url] | Node.js | MIT | [Redis][redis-url] | ![Stars][node-redis-stars] |
| [nredisstack][nredisstack-url] | .NET | MIT | [Redis][redis-url] | ![Stars][nredisstack-stars] |
| [redisearch-go][redisearch-go-url] | Go | BSD | [Redis][redisearch-go-author] | [![redisearch-go-stars]][redisearch-go-url] |
| [redisearch-api-rs][redisearch-api-rs-url] | Rust | BSD | [Redis][redisearch-api-rs-author] | [![redisearch-api-rs-stars]][redisearch-api-rs-url] |

[redis-url]: https://redis.com

[redis-py-url]: https://github.com/redis/redis-py
[redis-py-stars]: https://img.shields.io/github/stars/redis/redis-py.svg?style=social&amp;label=Star&amp;maxAge=2592000
[redis-py-package]: https://pypi.python.org/pypi/redis

[jedis-url]: https://github.com/redis/jedis
[jedis-stars]: https://img.shields.io/github/stars/redis/jedis.svg?style=social&amp;label=Star&amp;maxAge=2592000
[Jedis-package]: https://search.maven.org/artifact/redis.clients/jedis

[nredisstack-url]: https://github.com/redis/nredisstack
[nredisstack-stars]: https://img.shields.io/github/stars/redis/nredisstack.svg?style=social&amp;label=Star&amp;maxAge=2592000
[nredisstack-package]: https://www.nuget.org/packages/nredisstack/

[node-redis-url]: https://github.com/redis/node-redis
[node-redis-stars]: https://img.shields.io/github/stars/redis/node-redis.svg?style=social&amp;label=Star&amp;maxAge=2592000
[node-redis-package]: https://www.npmjs.com/package/redis

[redis-om-python-url]: https://github.com/redis/redis-om-python
[redis-om-python-author]: https://redis.com
[redis-om-python-stars]: https://img.shields.io/github/stars/redis/redis-om-python.svg?style=social&amp;label=Star&amp;maxAge=2592000

[redisearch-go-url]: https://github.com/RediSearch/redisearch-go
[redisearch-go-author]: https://redis.com
[redisearch-go-stars]: https://img.shields.io/github/stars/RediSearch/redisearch-go.svg?style=social&amp;label=Star&amp;maxAge=2592000

[redisearch-api-rs-url]: https://github.com/RediSearch/redisearch-api-rs
[redisearch-api-rs-author]: https://redis.com
[redisearch-api-rs-stars]: https://img.shields.io/github/stars/RediSearch/redisearch-api-rs.svg?style=social&amp;label=Star&amp;maxAge=2592000


以下のセルでは、Redisをベクターデータベースとして使用する方法を説明します。多くの方がすでにRedis APIに慣れ親しんでいると思われるため、これはほとんどの方にとって馴染みやすいはずです。

## セットアップ

RediSearchを使用してRedisをデプロイする方法は多数あります。最も簡単に始める方法はDockerを使用することですが、デプロイメントには多くの潜在的なオプションがあります。他のデプロイメントオプションについては、このリポジトリの[redisディレクトリ](./redis)を参照してください。

このチュートリアルでは、Docker上でRedis Stackを使用します。

以下のdockerコマンドを実行して、RediSearch付きのRedis（Redis Stack）のバージョンを開始します

```bash
$ cd redis
$ docker compose up -d
```
これには、Redisデータベースを管理するための[RedisInsight](https://redis.com/redis-enterprise/redis-insight/) GUIも含まれており、dockerコンテナを開始すると[http://localhost:8001](http://localhost:8001)で表示できます。

これで全ての設定が完了し、準備が整いました！次に、先ほど作成したRedisデータベースと通信するためのクライアントをインポートして作成します。

In [6]:
import redis
from redis.commands.search.indexDefinition import (
    IndexDefinition,
    IndexType
)
from redis.commands.search.query import Query
from redis.commands.search.field import (
    TextField,
    VectorField
)

REDIS_HOST =  "localhost"
REDIS_PORT = 6379
REDIS_PASSWORD = "" # default for passwordless Redis

# Connect to Redis
redis_client = redis.Redis(
    host=REDIS_HOST,
    port=REDIS_PORT,
    password=REDIS_PASSWORD
)
redis_client.ping()

True

## 検索インデックスの作成

以下のセルでは、Redisで検索インデックスを指定し、作成する方法を示します。以下の手順で進めます：

1. 距離メトリックやインデックス名など、インデックスを定義するための定数を設定する
2. RediSearchフィールドを使用してインデックススキーマを定義する
3. インデックスを作成する

In [7]:
# Constants
VECTOR_DIM = len(article_df['title_vector'][0]) # length of the vectors
VECTOR_NUMBER = len(article_df)                 # initial number of vectors
INDEX_NAME = "embeddings-index"                 # name of the search index
PREFIX = "doc"                                  # prefix for the document keys
DISTANCE_METRIC = "COSINE"                      # distance metric for the vectors (ex. COSINE, IP, L2)

In [8]:
# Define RediSearch fields for each of the columns in the dataset
title = TextField(name="title")
url = TextField(name="url")
text = TextField(name="text")
title_embedding = VectorField("title_vector",
    "FLAT", {
        "TYPE": "FLOAT32",
        "DIM": VECTOR_DIM,
        "DISTANCE_METRIC": DISTANCE_METRIC,
        "INITIAL_CAP": VECTOR_NUMBER,
    }
)
text_embedding = VectorField("content_vector",
    "FLAT", {
        "TYPE": "FLOAT32",
        "DIM": VECTOR_DIM,
        "DISTANCE_METRIC": DISTANCE_METRIC,
        "INITIAL_CAP": VECTOR_NUMBER,
    }
)
fields = [title, url, text, title_embedding, text_embedding]

In [9]:
# Check if index exists
try:
    redis_client.ft(INDEX_NAME).info()
    print("Index already exists")
except:
    # Create RediSearch Index
    redis_client.ft(INDEX_NAME).create_index(
        fields = fields,
        definition = IndexDefinition(prefix=[PREFIX], index_type=IndexType.HASH)
    )

## インデックスにドキュメントを読み込む

検索インデックスが作成できたので、ドキュメントを読み込むことができます。前の例で使用したのと同じドキュメントを使用します。Redisでは、ドキュメントの保存にHashまたはJSON（RediSearchに加えてRedisJSONを使用する場合）データタイプを使用できます。この例ではHASHデータタイプを使用します。以下のセルでは、インデックスにドキュメントを読み込む方法を示します。

In [10]:
def index_documents(client: redis.Redis, prefix: str, documents: pd.DataFrame):
    records = documents.to_dict("records")
    for doc in records:
        key = f"{prefix}:{str(doc['id'])}"

        # create byte vectors for title and content
        title_embedding = np.array(doc["title_vector"], dtype=np.float32).tobytes()
        content_embedding = np.array(doc["content_vector"], dtype=np.float32).tobytes()

        # replace list of floats with byte vectors
        doc["title_vector"] = title_embedding
        doc["content_vector"] = content_embedding

        client.hset(key, mapping = doc)

In [11]:
index_documents(redis_client, PREFIX, article_df)
print(f"Loaded {redis_client.info()['db0']['keys']} documents in Redis search index with name: {INDEX_NAME}")

Loaded 25000 documents in Redis search index with name: embeddings-index


## 検索クエリの実行

検索インデックスを作成し、ドキュメントを読み込んだので、検索クエリを実行できます。以下では、検索クエリを実行して結果を返す関数を提供します。この関数を使用して、Redisをベクトルデータベースとして活用する方法を示すいくつかのクエリを実行します。各例では、Redisで検索アプリケーションを開発する際に留意すべき特定の機能を実演します。

1. **返却フィールド**: 検索結果で返したいフィールドを指定できます。これは、ドキュメント内のフィールドのサブセットのみを返したい場合に便利で、ドキュメントを取得するための別の呼び出しを必要としません。以下の例では、検索結果で`title`フィールドのみを返します。
2. **ハイブリッド検索**: ベクトル検索を、全文検索、タグ、地理的検索、数値検索などの他のRediSearchフィールドと組み合わせて、ハイブリッド検索を行うことができます。以下の例では、ベクトル検索と全文検索を組み合わせます。

In [12]:
def search_redis(
    redis_client: redis.Redis,
    user_query: str,
    index_name: str = "embeddings-index",
    vector_field: str = "title_vector",
    return_fields: list = ["title", "url", "text", "vector_score"],
    hybrid_fields = "*",
    k: int = 20,
) -> List[dict]:

    # Creates embedding vector from user query
    embedded_query = openai.Embedding.create(input=user_query,
                                            model=EMBEDDING_MODEL,
                                            )["data"][0]['embedding']

    # Prepare the Query
    base_query = f'{hybrid_fields}=>[KNN {k} @{vector_field} $vector AS vector_score]'
    query = (
        Query(base_query)
         .return_fields(*return_fields)
         .sort_by("vector_score")
         .paging(0, k)
         .dialect(2)
    )
    params_dict = {"vector": np.array(embedded_query).astype(dtype=np.float32).tobytes()}

    # perform vector search
    results = redis_client.ft(index_name).search(query, params_dict)
    for i, article in enumerate(results.docs):
        score = 1 - float(article.vector_score)
        print(f"{i}. {article.title} (Score: {round(score ,3) })")
    return results.docs

In [13]:
# For using OpenAI to generate query embedding
openai.api_key = os.getenv("OPENAI_API_KEY", "sk-xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx")
results = search_redis(redis_client, 'modern art in Europe', k=10)

0. Museum of Modern Art (Score: 0.875)
1. Western Europe (Score: 0.867)
2. Renaissance art (Score: 0.864)
3. Pop art (Score: 0.86)
4. Northern Europe (Score: 0.855)
5. Hellenistic art (Score: 0.853)
6. Modernist literature (Score: 0.847)
7. Art film (Score: 0.843)
8. Central Europe (Score: 0.843)
9. European (Score: 0.841)


In [14]:
results = search_redis(redis_client, 'Famous battles in Scottish history', vector_field='content_vector', k=10)

0. Battle of Bannockburn (Score: 0.869)
1. Wars of Scottish Independence (Score: 0.861)
2. 1651 (Score: 0.853)
3. First War of Scottish Independence (Score: 0.85)
4. Robert I of Scotland (Score: 0.846)
5. 841 (Score: 0.844)
6. 1716 (Score: 0.844)
7. 1314 (Score: 0.837)
8. 1263 (Score: 0.836)
9. William Wallace (Score: 0.835)


## Redisを使用したハイブリッドクエリ

前の例では、RediSearchを使用してベクトル検索クエリを実行する方法を示しました。このセクションでは、ハイブリッド検索のためにベクトル検索を他のRediSearchフィールドと組み合わせる方法を説明します。以下の例では、ベクトル検索と全文検索を組み合わせます。

In [15]:
def create_hybrid_field(field_name: str, value: str) -> str:
    return f'@{field_name}:"{value}"'

In [16]:
# search the content vector for articles about famous battles in Scottish history and only include results with Scottish in the title
results = search_redis(redis_client,
                       "Famous battles in Scottish history",
                       vector_field="title_vector",
                       k=5,
                       hybrid_fields=create_hybrid_field("title", "Scottish")
                       )

0. First War of Scottish Independence (Score: 0.892)
1. Wars of Scottish Independence (Score: 0.889)
2. Second War of Scottish Independence (Score: 0.879)
3. List of Scottish monarchs (Score: 0.873)
4. Scottish Borders (Score: 0.863)


In [17]:
# run a hybrid query for articles about Art in the title vector and only include results with the phrase "Leonardo da Vinci" in the text
results = search_redis(redis_client,
                       "Art",
                       vector_field="title_vector",
                       k=5,
                       hybrid_fields=create_hybrid_field("text", "Leonardo da Vinci")
                       )

# find specific mention of Leonardo da Vinci in the text that our full-text-search query returned
mention = [sentence for sentence in results[0].text.split("\n") if "Leonardo da Vinci" in sentence][0]
mention

0. Art (Score: 1.0)
1. Paint (Score: 0.896)
2. Renaissance art (Score: 0.88)
3. Painting (Score: 0.874)
4. Renaissance (Score: 0.846)


'In Europe, after the Middle Ages, there was a "Renaissance" which means "rebirth". People rediscovered science and artists were allowed to paint subjects other than religious subjects. People like Michelangelo and Leonardo da Vinci still painted religious pictures, but they also now could paint mythological pictures too. These artists also invented perspective where things in the distance look smaller in the picture. This was new because in the Middle Ages people would paint all the figures close up and just overlapping each other. These artists used nudity regularly in their art.'

Redis をベクターデータベースとして使用するより多くの例については、このリポジトリの ``vector_databases/redis`` ディレクトリ内の README と例を参照してください。